# VidGen 🎬 - AI-Powered Video Generation in Google Colab

This notebook demonstrates how to use VidGen to create AI-generated videos from text scripts using Google Colab's free GPU resources.

## 🚀 Features
- **AI Script Processing**: Powered by Google Gemini
- **Image Generation**: Stable Diffusion for character images
- **Voice Synthesis**: Bark TTS for natural speech
- **Audio Generation**: Procedural background music and effects
- **Video Assembly**: Professional transitions and effects

## 📋 Prerequisites
1. **GPU Runtime**: Enable GPU in Runtime → Change runtime type
2. **API Key**: Get a Google Gemini API key from [Google AI Studio](https://makersuite.google.com/)

---

## 1. 📦 Setup and Installation

Run this cell to install VidGen and all dependencies in Colab.

In [ ]:
# Check GPU availability
!nvidia-smi

# Clone VidGen repository
!git clone https://github.com/AhmedWGabr/VidGen.git
%cd VidGen

# Run Colab setup script
!python setup_colab.py

## 2. 🔑 Configure API Keys

Set up your Google Gemini API key for script processing.

In [ ]:
import os
from google.colab import userdata

# Option 1: Use Colab Secrets (Recommended)
try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    print("✅ API key loaded from Colab secrets")
except:
    # Option 2: Manual input (less secure)
    from getpass import getpass
    GEMINI_API_KEY = getpass("Enter your Gemini API Key: ")
    print("✅ API key entered manually")

# Set environment variable
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY
print("🔑 API key configured successfully")

## 3. 🎬 Generate Your First Video

Let's create a simple video from a text script.

In [ ]:
from vidgen.main import main as vidgen_main
from vidgen.core.config import VideoGenConfig
import tempfile
import os

# Create a sample script
sample_script = """
Title: "AI in the Future"

A futuristic laboratory with glowing screens and holographic displays.
[Background: Ambient electronic music]

NARRATOR: "Welcome to the year 2030, where artificial intelligence has transformed our world."

A scientist in a white lab coat examining a holographic brain model.

SCIENTIST: "We've made incredible breakthroughs in understanding consciousness."

Close-up of advanced AI neural networks visualized as flowing data streams.

NARRATOR: "The future is here, and it's more amazing than we ever imagined."
"""

# Save script to file
script_path = "/content/vidgen/sample_script.txt"
with open(script_path, 'w') as f:
    f.write(sample_script)

print("📝 Sample script created")
print("🎬 Starting video generation...")

# Generate video
try:
    output_path = vidgen_main(
        script_file=script_path,
        api_key=GEMINI_API_KEY,
        segment_duration=5
    )
    print(f"✅ Video generated successfully: {output_path}")
except Exception as e:
    print(f"❌ Error generating video: {e}")

## 4. 🎨 Advanced Video Generation

Use the advanced API for more control over video generation.

In [ ]:
from vidgen.services.script_parser import ScriptParser
from vidgen.services.gemini_api import call_gemini_api
from vidgen.models.video import generate_video_segment
from vidgen.models.audio import generate_background_audio
from vidgen.services.video_assembler import assemble_video
import json

# Advanced script with multiple characters
advanced_script = """
Title: "The Digital Revolution"

A bustling tech startup office with developers coding.
[Background: Upbeat tech music]

ALEX: "We're on the verge of something revolutionary."

Close-up of code streaming across multiple monitors.

SARAH: "This AI will change how we interact with technology forever."

A sleek AI interface with flowing data visualizations.

NARRATOR: "Innovation never sleeps in the digital age."
"""

# Process script with Gemini
print("🤖 Processing script with Gemini AI...")
gemini_response = call_gemini_api(advanced_script, GEMINI_API_KEY, segment_duration=6)

if gemini_response:
    # Parse the structured response
    parser = ScriptParser()
    parsed_script = parser.parse_gemini_response(gemini_response)
    
    print(f"📊 Parsed {len(parsed_script)} video segments")
    
    # Generate video segments
    video_segments = []
    face_cache = {}  # Cache character faces
    
    for i, segment in enumerate(parsed_script):
        print(f"🎬 Generating segment {i+1}/{len(parsed_script)}...")
        
        segment_path = generate_video_segment(
            segment, 
            face_cache=face_cache,
            default_duration=6
        )
        
        if segment_path:
            video_segments.append(segment_path)
            print(f"✅ Segment {i+1} completed")
    
    # Generate background audio
    print("🎵 Generating background audio...")
    bg_audio = generate_background_audio("Upbeat tech music")
    
    # Assemble final video
    print("🎞️ Assembling final video...")
    final_video = assemble_video(
        video_segments=video_segments,
        background_audio=bg_audio,
        output_filename="advanced_video.mp4"
    )
    
    print(f"🎉 Advanced video completed: {final_video}")
else:
    print("❌ Failed to process script with Gemini")

## 5. 🖥️ Launch Web Interface

Use the Gradio web interface for interactive video generation.

In [ ]:
from vidgen.ui.gradio_app import create_gradio_interface

# Create and launch Gradio interface
interface = create_gradio_interface()

# Launch with public link for Colab
interface.launch(
    share=True,  # Creates public link
    debug=True,  # Enable debugging
    server_name="0.0.0.0",  # Allow external connections
    server_port=7860  # Default Gradio port
)

## 6. 📁 Download Generated Videos

Download your generated videos to your local machine.

In [ ]:
from google.colab import files
import os
import glob

# List all generated videos
video_dir = "/content/vidgen/outputs/videos"
video_files = glob.glob(os.path.join(video_dir, "*.mp4"))

print(f"📹 Found {len(video_files)} generated videos:")
for video in video_files:
    print(f"  - {os.path.basename(video)}")

# Download all videos
if video_files:
    print("\n⬇️ Downloading videos...")
    for video in video_files:
        try:
            files.download(video)
            print(f"✅ Downloaded: {os.path.basename(video)}")
        except Exception as e:
            print(f"❌ Failed to download {video}: {e}")
else:
    print("No videos found. Generate some videos first!")

## 7. 🧪 Test Individual Components

Test individual VidGen components for debugging and experimentation.

In [ ]:
# Test image generation
print("🖼️ Testing image generation...")
from vidgen.models.image import generate_character_image

image_path = generate_character_image(
    "A friendly scientist in a white lab coat, professional headshot",
    seed=42
)
print(f"Image generated: {image_path}")

# Test TTS generation
print("\n🗣️ Testing text-to-speech...")
from vidgen.models.tts import generate_tts_audio

tts_path = generate_tts_audio("Hello, this is a test of the text to speech system.")
print(f"TTS audio generated: {tts_path}")

# Test audio generation
print("\n🎵 Testing background audio generation...")
from vidgen.models.audio import generate_background_audio

audio_path = generate_background_audio("Ambient forest sounds with gentle bird calls")
print(f"Background audio generated: {audio_path}")

print("\n✅ All components tested successfully!")

## 8. 📊 Performance Monitoring

Monitor GPU usage and performance metrics.

In [ ]:
import torch
import psutil
import time

def monitor_resources():
    """Monitor system resources during video generation."""
    
    # GPU information
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
        gpu_memory_used = torch.cuda.memory_allocated(0) / 1e9
        gpu_memory_cached = torch.cuda.memory_reserved(0) / 1e9
        
        print(f"🔥 GPU: {torch.cuda.get_device_name(0)}")
        print(f"📊 GPU Memory: {gpu_memory_used:.1f}GB / {gpu_memory:.1f}GB")
        print(f"💾 GPU Cached: {gpu_memory_cached:.1f}GB")
    else:
        print("❌ No GPU available")
    
    # CPU and RAM information
    cpu_percent = psutil.cpu_percent(interval=1)
    memory = psutil.virtual_memory()
    
    print(f"💻 CPU Usage: {cpu_percent:.1f}%")
    print(f"🧠 RAM Usage: {memory.percent:.1f}% ({memory.used/1e9:.1f}GB / {memory.total/1e9:.1f}GB)")
    
    # Disk space
    disk = psutil.disk_usage('/content')
    print(f"💾 Disk Usage: {disk.percent:.1f}% ({disk.used/1e9:.1f}GB / {disk.total/1e9:.1f}GB)")

print("📊 Current System Status:")
monitor_resources()

# Clear GPU cache if needed
if torch.cuda.is_available():
    print("\n🧹 Clearing GPU cache...")
    torch.cuda.empty_cache()
    print("✅ GPU cache cleared")

## 🎯 Tips for Best Results in Colab

### 🚀 Performance Tips
1. **Use GPU Runtime**: Always enable GPU for faster generation
2. **Monitor Memory**: Check GPU memory usage to avoid OOM errors
3. **Cache Models**: Models are cached automatically to speed up subsequent runs
4. **Batch Processing**: Generate multiple segments efficiently

### 📝 Script Writing Tips
1. **Be Descriptive**: Detailed scene descriptions produce better images
2. **Character Consistency**: Use consistent character descriptions
3. **Audio Cues**: Include background music and sound effect descriptions
4. **Pacing**: Aim for 5-10 second segments for best results

### 🔧 Troubleshooting
- **Out of Memory**: Restart runtime and clear GPU cache
- **Slow Generation**: Check GPU is enabled and models are cached
- **API Errors**: Verify Gemini API key is correctly set
- **Audio Issues**: Ensure FFmpeg is properly installed

---

**🎉 Happy Video Creating with VidGen! 🎬**